In [68]:
import requests
from bs4 import BeautifulSoup
import tqdm
import time
import numpy as np
import pandas as pd
import nltk   
import unicodedata
from html.parser import HTMLParser
import re

saved_links = pd.read_csv('ricardoeletro.csv')
true_links = np.array(saved_links[saved_links['label']==1]['url'])
url  = true_links[0]
code = requests.get(url)
plain = code.text

In [108]:
def to_ascii(string):
    latin_dict = \
        {'\\u00e1':'a', '\\u00e0':'a', '\\u00e2':'a', '\\u00e3':'a', '\\u00e4':'a', '\\u00c1':'A', '\\u00c0':'A', 
         '\\u00c2':'A', '\\u00c3':'A', '\\u00c4':'A', '\\u00e9':'e', '\\u00e8':'e', '\\u00ea':'e', '\\u00ea':'e', 
         '\\u00c9':'E', '\\u00c8':'E', '\\u00ca':'E', '\\u00cb':'E', '\\u00ed':'i', '\\u00ec':'i', '\\u00ee':'i', 
         '\\u00ef':'i', '\\u00cd':'I', '\\u00cc':'I', '\\u00ce':'I', '\\u00cf':'I', '\\u00f3':'o', '\\u00f2':'o', 
         '\\u00f4':'o', '\\u00f5':'o', '\\u00f6':'o', '\\u00d3':'O', '\\u00d2':'O', '\\u00d4':'O', '\\u00d5':'O', 
         '\\u00d6':'O', '\\u00fa':'u', '\\u00f9':'u', '\\u00fb':'u', '\\u00fc':'u', '\\u00da':'U', '\\u00d9':'U', 
         '\\u00db':'U', '\\u00e7':'c', '\\u00c7':'C', '\\u00f1':'n', '\\u00d1':'N', '\\u0026':'E', '\\u0027':'\'',
         'á':'a', 'à':'a', 'â':'a', 'ã':'a', 'ä':'a', 'Á':'A', 'À':'A', 'Â':'A', 'Ã':'A', 'Ä':'A', 'é':'e', 'è':'e', 
         'ê':'e', 'ê':'e', 'É':'E', 'È':'E', 'Ê':'E', 'Ë':'E', 'í':'i', 'ì':'i', 'î':'i', 'ï':'i', 'Í':'I', 'Ì':'I', 
         'Î':'I', 'Ï':'I', 'ó':'o', 'ò':'o', 'ô':'o', 'õ':'o', 'ö':'o', 'Ó':'O', 'Ò':'O', 'Ô':'O', 'Õ':'O', 'Ö':'O', 
         'ú':'u', 'ù':'u', 'û':'u', 'ü':'u', 'Ú':'U', 'Ù':'U', 'Û':'U', 'ç':'c', 'Ç':'C', 'ñ':'n', 'Ñ':'N', '&':'E'}
    for key in latin_dict:
        string = string.replace(key, latin_dict[key])
    return string

def html_to_np(table):
    result = []
    all_rows = table.find_all('tr')
    for row in all_rows:
        result.append([])
        all_cols = row.find_all('td')
        for col in all_cols:
            col_text = [s for s in col.find_all(text=True)]
            col_text  = ''.join(col_text) # it should not change anything
            result[-1].append(col_text)
    return np.array(result)



In [109]:
saved_links.tail()

,url,label
198,https://www.ricardoeletro.com.br/Produto/Smart...,1
199,https://www.ricardoeletro.com.br/Produto/Smart...,1
200,https://www.ricardoeletro.com.br/Produto/Smart...,1
201,https://www.ricardoeletro.com.br/Produto/Smart...,1
202,https://www.ricardoeletro.com.br/Televendas,0


In [110]:
#plain = (unicodedata.normalize('NFD', plain).encode('ascii', 'ignore'))
s = BeautifulSoup(plain, "html.parser")
plain = to_ascii(s.decode())
s = BeautifulSoup(plain, "html.parser")

In [111]:
head_text = s.head.title.find_all(text=True)
head_text

['Celular DL YC335 Azul, Flip, Dual Chip, Tela de 1.8, Camera,']

In [112]:
table = s.find_all("table")[0]
table = max(s.find_all("table"),key=len)

np_table = (html_to_np(table))
att, values = np_table[:,0], np_table[:,1]

In [113]:
X = ' '.join(head_text) + ' '+ ' '.join(att)
X

'Celular DL YC335 Azul, Flip, Dual Chip, Tela de 1.8, Camera, Camera Traseira Toques Visualizador de arquivos Suporte a GPS SCREEN_TECHNOLOGY Frequencias NUMBER_OF_SIM_CARD_SLOTS INTERNAL_MEMORY Relsoucao do visor/display interno (pixels) Tamanho do visor (cm) HSDPA (maior velocidade em download) Sincronizacao com PC TV-Out WIFI_CONNECTION Touch Screen Color RAM Dual Chip OPERATING_SYSTEM Formato de Audio FRONT_CAMERA_RESOLUTION Grava video Slot para cartao Cartao Incluso Mensagens Altura Largura Comprimento Dimensoes aproximadas da embalagem (AxLxC) Peso Garantia'

In [117]:
regex = re.compile('[^a-zA-Z ]')
X = regex.sub('', X)
X

'Celular DL YC Azul Flip Dual Chip Tela de  Camera Camera Traseira Toques Visualizador de arquivos Suporte a GPS SCREENTECHNOLOGY Frequencias NUMBEROFSIMCARDSLOTS INTERNALMEMORY Relsoucao do visordisplay interno pixels Tamanho do visor cm HSDPA maior velocidade em download Sincronizacao com PC TVOut WIFICONNECTION Touch Screen Color RAM Dual Chip OPERATINGSYSTEM Formato de Audio FRONTCAMERARESOLUTION Grava video Slot para cartao Cartao Incluso Mensagens Altura Largura Comprimento Dimensoes aproximadas da embalagem AxLxC Peso Garantia'

In [132]:
X = [ word.lower() for word in  X.split(' ') if len(word) > 2]
X = sorted(list(set(X)))
X

['altura',
 'aproximadas',
 'arquivos',
 'audio',
 'axlxc',
 'azul',
 'camera',
 'cartao',
 'celular',
 'chip',
 'color',
 'com',
 'comprimento',
 'dimensoes',
 'download',
 'dual',
 'embalagem',
 'flip',
 'formato',
 'frequencias',
 'frontcameraresolution',
 'garantia',
 'gps',
 'grava',
 'hsdpa',
 'incluso',
 'internalmemory',
 'interno',
 'largura',
 'maior',
 'mensagens',
 'numberofsimcardslots',
 'operatingsystem',
 'para',
 'peso',
 'pixels',
 'ram',
 'relsoucao',
 'screen',
 'screentechnology',
 'sincronizacao',
 'slot',
 'suporte',
 'tamanho',
 'tela',
 'toques',
 'touch',
 'traseira',
 'tvout',
 'velocidade',
 'video',
 'visor',
 'visordisplay',
 'visualizador',
 'wificonnection']